In [1]:
from appgeopy import *
from my_packages import *

In [ ]:
# Extract necessary parameters
prefix = "Choushui"
mainfolder = r"D:\1000_SCRIPTS\002_Manuscript_1"
o_displacement_fpath = os.path.join(mainfolder, "17_VERTICAL", "CALIBRATED_oChoushui_DISPLACEMENT_VERT_mm.pkl")
gps_shapefile_fpath = r"E:\006_InSAR_Project_2022\GPSDataWorkspace\Selected_ActiveGPS_4Calibration_20220609.shp"
x_col = "X_TWD97"
y_col = "Y_TWD97"
epsg = "EPSG:3826"
radius_of_buffer = 1000

# Load displacement data and convert to GeoDataFrame
o_displacement_df = pd.read_pickle(o_displacement_fpath, compression="zip")
o_displacement_geodf = geospatial.convert_to_geodata(df=o_displacement_df, xcoord_col=x_col, ycoord_col=y_col, crs_epsg=epsg)

# Load GPS shapefile
gps_shapefile_geodf = gpd.read_file(gps_shapefile_fpath)

# Find Persistent Scatterers (PSC) near each GPS station
cache = [
    geospatial.find_point_neighbors(row, o_displacement_geodf, "STATION", radius_of_buffer)
    for idx, row in gps_shapefile_geodf.iterrows()
]

# Combine results into a single DataFrame and save
nearby_psc_pointkey = pd.concat(cache, ignore_index=True).loc[:, ["PointKey", "X_TWD97", "Y_TWD97", "STATION"]]
output_savepath = os.path.join(mainfolder, "17_VERTICAL", f"o{prefix}_PSC_nearbyGPS_{radius_of_buffer}m.pkl")
nearby_psc_pointkey.to_pickle(output_savepath, compression="zip")
